In [1]:
import tensorflow as tf

2025-01-10 10:39:37.521256: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


FFN: Point-wise Feed-Forward Networks
Point-wise指的是FFN逐词对token做变换，即便实现上没有给每个token各自保留一份FFN
FFN实现上是以词矩阵为输入，而不是词向量，下面比较两者在计算上是否完全等价
FFN和attention的关系：attention捕捉token之间的关系，侧重全局关系；FFN独立对每个token变换，侧重局部关系。互为补充
FFN的权重在token之间共享：参数效率和泛化

In [31]:
X = tf.constant(1, shape=[10,8], dtype=tf.int32)
W = tf.random.uniform(shape=[8,12], minval=0, maxval=2, dtype=tf.int32)
def whole_FFN(X, W):
    return tf.nn.relu(tf.matmul(X, W))
def individual_FFN(X, W):
    # X [10,8]
    x_list = tf.unstack(X, axis=0) # 10 * [8]
    x_list = [tf.expand_dims(x, axis=0) for x in x_list] # 10 * [1, 8]
    splited_output = [tf.nn.relu(tf.matmul(x, W)) for x in x_list] 
    # list中的元素是[1, dim]，如果是直接stack，结果是[n, 1, dim]，而非[n, dim]
    concated_output = tf.stack(splited_output, axis=0) # [10, 1, 8]
    shaped_output = tf.squeeze(concated_output, axis=1) # [10, 8]
    return shaped_output
tf.reduce_all( ( whole_FFN(X, W) == individual_FFN(X, W))) # true为完全相同

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [32]:
X = tf.random.normal(shape=[2, 2])
W = tf.random.normal(shape=[2, 2])
tf.reduce_all( ( whole_FFN(X, W) == individual_FFN(X, W))) # true为完全相同

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [33]:
from tensorflow.keras.layers import Layer, Dense

In [38]:
class FFN(Layer):
    def __init__(self, d_model, dff):
        super(FFN, self).__init__()
        self.w1 = Dense(dff, activation="relu")
        self.w2 = Dense(d_model)
    def call(self, X):
        return self.w2(self.w1(X))

In [39]:
X = tf.random.normal([10,512])
d_model = 512
dff = 2048
ffn = FFN(512, 2048)
ffn(X)

<tf.Tensor: shape=(10, 512), dtype=float32, numpy=
array([[ 0.58470833, -0.28071082,  0.33368957, ..., -0.48978215,
        -0.47320676,  0.78683853],
       [-0.25092518, -0.38057712, -0.39177847, ..., -0.458938  ,
        -0.9278326 , -0.3361459 ],
       [-0.55137634,  0.27648982, -0.2708023 , ..., -0.13214004,
        -0.6609394 ,  0.5344621 ],
       ...,
       [-0.5541176 ,  0.24510771, -0.18587717, ..., -0.80350685,
        -0.42426658,  0.6435993 ],
       [ 0.21257944, -0.60036874, -0.17853856, ..., -0.23341371,
         0.28404212,  0.13091493],
       [ 0.67460966, -0.41883776, -0.27480543, ..., -0.3294828 ,
        -0.11506021,  0.4455172 ]], dtype=float32)>